# SWOP script 4
## Compute mean amplitude measurements and export for analysis in R

Analysis script associated with the manuscript: ***Native word order processing is not uniform: An ERP-study of verb-second word order***, by Susan Sayehli, Marianne Gullberg, Aaron Newman, and Annika Andersson. Currently under review with *Frontiers in Psychology - Language Sciences*, manuscript 668276.


This notebook reads in the *-epo.fif evoked files from individual subjects that were produced by the batch individual subject preprocessing in the script *SWOP 1 - preprocessing.py*

This script does not compute or apply baseline regression to the data. Rather, it computes mean amplitudes in specified time windows, including the prestimulus baseline period (-100-0 ms). Then, the baseline mean amplitude is added as a column/variable for each subject/trial/electrode, so that baseline can be used as a variable in mixed-effects analysis.

---
Copyright 2016-21  [Aaron J Newman](https://github.com/aaronjnewman), [NeuroCognitive Imaging Lab](http://ncil.science), [Dalhousie University](https://dal.ca)

Released under the [The 3-Clause BSD License](https://opensource.org/licenses/BSD-3-Clause)

---

## Load in the necessary libraries/packages we'll need

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)
from scipy.stats import zscore

import mne
mne.set_log_level(verbose='error')

## List of subjects

In [2]:
subjects = ['s_04nm',  's_07ba',  's_09lo',  's_12wg',  's_13ff',  's_14mc',
            's_15rj',  's_17oh',  's_18ak',  's_19am',  's_21ma',  's_23nj',
            's_24zk',  's_25ks',  's_26nm',  's_27lm',  's_28js',  's_29ld',
            's_30la',  's_31bf']

## Time windows of interest

In [3]:
# Define the traditional baseline window
baseline = (-.100, 0.)
tmin, tmax = -.100, 1.
# Define start & end of time window to compute MeanAmplitude over
time_wins = [(.100, .300), (.300, .500), (.500, .700), (.700, .900), (.900, 1.000)]

## Conditions and Contrasts of Interest

In [4]:
target_words = ['kanske', 'hemma', 'idag']

conditions = ['V2/kanske', 'V3/kanske', 
              'V2/hemma',  'V3/hemma',
              'V2/idag',   'V3/idag'
             ]

contrasts = {'kanske':['V3/kanske', 'V2/kanske'],
             'hemma':['V3/hemma',   'V2/hemma'],
             'idag':['V3/idag',     'V2/idag'],             
             }

## Time windows

In [5]:
components = {'baseline':{'tw':(-.100, 0.), 'tw_width':.100},
              '100-300':{'tw':(.100, .300), 'tw_width':.200},
              '300-500':{'tw':(.300, .500), 'tw_width':.200},
              '500-700':{'tw':(.500, .700), 'tw_width':.200},
              '700-900':{'tw':(.700, .900), 'tw_width':.200},
              '900-1000':{'tw':(.900, 1.), 'tw_width':.100},
              }

## Define ROIs
clusters of electrodes to average over for waveform plots

In [6]:
rois = {'L_Ant':['F7', 'FT7', 'F3', 'FC3'],
        'M_Ant':['Fz'],
        'R_Ant':['F8', 'FT8', 'F4', 'FC4'],
        'L_Cent':['T7', 'TP7', 'C3', 'CP3'], 
        'M_Cent':['Cz'], 
        'R_Cent':['T8', 'TP8', 'C4', 'CP4'],
        'L_Post':['P7', 'P3', 'PO7', 'O1'],
        'M_Post':['Pz'],
        'R_Post':['P8', 'P4', 'PO8', 'O2'],
        'Other':['Fp1', 'Fp2', 'M1', 'M2']
       }

roi_order = ['L_Ant', 'M_Ant', 'R_Ant',
             'L_Cent', 'M_Cent', 'R_Cent',
             'L_Post', 'M_Post', 'R_Post'
            ]

ch_roi_map = {}
for roi, chans in rois.items():
    for chan in chans:
        ch_roi_map[chan] = roi

# We don't need Other anymore
rois.pop('Other');   

---
## Read in the data

In [7]:
data_path = '../data/'

epochs = {}
for subject in subjects:
    epochs[subject] = mne.read_epochs(str(data_path + subject + '-epo.fif'),
                                         proj=False, 
                                         verbose=None, 
                                         preload=True).set_eeg_reference(ref_channels=['M1', 'M2'])

---
## Compute single-trial measurements

In [ ]:
%%time

df_list = []
metadata_cols = ['Sentence', 'postphrase', 'pronoun_noun', 'questiondisplay.ACC', 
                 'questiondisplay.RT', 'sentence_no', 'stimulussentence']


for subj in subjects:
    for cond in conditions:
        for roi, chans in rois.items():
            for comp_name, comp_params in components.items():

                win_start, win_stop = np.searchsorted(epochs[subj][cond].times, comp_params['tw'])

                df_list.append(pd.concat([pd.DataFrame({'Subject': subj, 
                                                        'Component':comp_name,
                                                        'Trial Time':np.repeat(epochs[subj][cond].events[:,0], len(chans)),
                                                        'Condition':cond,
                                                        'ROI':roi,
                                                        'Channel':np.tile(chans, epochs[subj][cond].selection.shape),
                                                        }),
                                            pd.DataFrame(epochs[subj][cond].get_data(picks=chans)[:, :, win_start:win_stop].mean(axis=-1).flatten() * 10e5,
                                                         columns=['MeanAmpl']), 
                                            pd.DataFrame(np.repeat(np.array(epochs[subj][cond].metadata[metadata_cols]).byteswap().newbyteorder(), len(chans), axis=0), 
                                                         columns=metadata_cols),
                                             ], 
                                             axis=1))

df = pd.concat(df_list)
df[['SentPos', 'Adverb']] = df['Condition'].str.split('/', expand=True)
df[['Laterality', 'AntPost']] = df['ROI'].str.split('_', expand=True)

## Plot histogram
look for outliers

In [ ]:
sns.displot(data=df,
            x='MeanAmpl', hue='Adverb')
plt.show()

## Remove Outliers

Defined within-subjects, as data points (individual trials/channels) with abs(*z*) > 2.5 

In [ ]:
z_thresh = 2.5 # cutoff for defining outliers, in SD

# Compute standard (z) scores 
df['MeanAmpl z'] = df.loc[:, ['Subject', 
                               'MeanAmpl']].groupby('Subject').transform(zscore)

len_orig = len(df)

# Drop outliers based on ±2.5 SD
df = df[(df['MeanAmpl z'] >= -z_thresh) & (df['MeanAmpl z'] <= z_thresh)]

n_dropped = len_orig - len(df)
print(str(round(((n_dropped / len_orig) * 100), 3)) + '% of data dropped as outliers based MeanAmplitude z +/-' + str(z_thresh))

In [ ]:
sns.displot(data=df,
            x='MeanAmpl', hue='Adverb', alpha=.25)
plt.show()

In [57]:
df.sample(12)

,Subject,Component,Trial Time,Condition,ROI,Channel,MeanAmpl,Sentence,postphrase,pronoun_noun,questiondisplay.ACC,questiondisplay.RT,sentence_no,stimulussentence,SentPos,Adverb,Laterality,AntPost,MeanAmpl z
73,s_07ba,500-700,1594447,V3/kanske,L_Post,P3,4.779292,210,3.0,N,0,539,20.0,Kanske pojken ramlade och blev ledsen.,V3,kanske,L,Post,0.791424
76,s_29ld,900-1000,607795,V3/kanske,R_Cent,T8,1.900788,176,1.0,P,1,589,30.0,Kanske hon sprang snabbast.,V3,kanske,R,Cent,0.554020
2,s_18ak,300-500,65773,V2/idag,L_Cent,C3,-0.818199,8,0.0,P,1,1809,7.0,Idag grät hon.,V2,idag,L,Cent,-0.139667
103,s_26nm,baseline,1352280,V2/idag,L_Post,O1,5.873272,343,4.0,P,1,387,14.0,Idag lekte hon med Anders och Erik.,V2,idag,L,Post,1.154466
71,s_14mc,baseline,732312,V2/idag,L_Ant,FC3,3.424911,242,3.0,P,1,408,23.0,Idag sjöng hon hemma i köket.,V2,idag,L,Ant,0.671852
71,s_28js,900-1000,756601,V2/kanske,L_Ant,FC3,3.534934,206,3.0,N,1,317,25.0,Kanske skrev flickan i sin bok.,V2,kanske,L,Ant,0.649421
54,s_19am,100-300,664337,V3/hemma,L_Post,PO7,-0.689842,161,2.0,P,1,869,4.0,Hemma hon betalade för bollen.,V3,hemma,L,Post,-0.067387
94,s_31bf,300-500,1237083,V2/idag,L_Cent,C3,1.242409,297,1.0,N,1,181,5.0,Idag dansade pojken runt.,V2,idag,L,Cent,0.324851
90,s_09lo,900-1000,1187091,V2/kanske,R_Cent,C4,-11.212596,215,2.0,P,1,170,23.0,Kanske sjöng han för Sara.,V2,kanske,R,Cent,-2.354476
25,s_19am,300-500,477860,V2/kanske,L_Post,P3,-3.374819,116,3.0,P,1,2264,16.0,Kanske låg han och läste tidningen.,V2,kanske,L,Post,-0.648684


In [ ]:
df.shape

## Make baseline a variable

Pivote 'Component' and then melt back to a Component column without baseline. `baseline` is now its own column

In [ ]:
df = df.pivot(index=['Subject', 'Trial Time', 'Condition', 'ROI', 'Channel', 
                        'Sentence', 'postphrase', 'pronoun_noun', 
                        'questiondisplay.ACC', 'questiondisplay.RT', 
                        'sentence_no', 'stimulussentence', 
                        'SentPos', 'Adverb', 'Laterality', 'AntPost', 
                        ], 
                 columns='Component', 
                 values='MeanAmpl'
                )

In [ ]:
df = df.reset_index().melt(id_vars=['Subject', 'Trial Time', 'Condition', 'ROI', 'Channel', 
                                            'Sentence', 'postphrase', 'pronoun_noun', 
                                            'questiondisplay.ACC', 'questiondisplay.RT', 
                                            'sentence_no', 'stimulussentence', 
                                            'SentPos', 'Adverb', 'Laterality', 'AntPost', 'baseline'
                                            ],
                                value_vars=['100-300', '300-500', '500-700', '700-900', '900-1000'],
                              ignore_index=False,
                                   value_name='MeanAmpl'
                                  )

In [ ]:
df.shape

## Confirm this worked
For a given subject/trial/electrode, the value in `baseline` should appear for every component time window.

In [ ]:
df[(df['Subject']=='s_18ak') & (df['Trial Time']==47855) & (df['Channel']=='Cz')]

In [ ]:
df[(df['Subject']=='s_18ak') & (df['Trial Time']==1021477) & (df['Channel']=='Cz')]

## Export to file

In [ ]:
df.to_csv('../group_data/SWOP_ERP_measurements.csv', index=False)